Подсчитать, сколько раз встречается каждая из русских букв в текстовых файлах (заглавные и строчные символы не различаются), лежащих в папке. Выполнить задание с использованием Dask.Bag.

In [ ]:
def concat_db_files(files):
    result_beg = db.from_sequence([])
    for file in files:
        result_beg = db.concat([result_beg, db.read_text(file, encoding='cp1251')])
    return result_beg

def isrus(x):
    return re.match("[а-яА-Я]", x)

In [ ]:
files = list(map(lambda x: 'book\\' + x, os.listdir('book')))
set_book = concat_db_files(files)

In [ ]:
result = set_book.map(lambda x: x.lower().strip()).flatten().filter(isrus).frequencies().compute()

Подсчитать, сколько раз встречается каждое из русских слов (слов, состоящих только из русских букв и знака "-") в текстовых файлах (заглавные и строчные символы не различаются, слова перенесенные через строки не рассматриваются), лежащих в папке.

In [ ]:
result = b.map(lambda x: re.sub('[!@#$.?:;"]',"",
                                x.lower().strip()).split()).flatten().filter(isrus).frequencies().compute()

Для одного из файлов с отзывами из https://yadi.sk/d/OUpjPvLX0i1Ipw подсчитать частоту (!) слов для отзывов с оценкой "5" и для отзывов с оценкой "2" и найти 50 распространенных слов (встречающихся в каждой из групп более 10 раз), частота которых максимально увеличивается и уменьшается при переходе от плохих отзывов к хорошим.

In [ ]:
rev_games = db.read_text('..\data\amazon-reviews\reviews_Video_Games_5.json.gz')
js = rev_games.map(json.loads)

In [ ]:
%%time
result_5 = js.filter(lambda record: record['overall'] == 5.0).\
            map(lambda x: re.sub('[!@#$.?:;()"]',"",x['reviewText'].lower().strip()).split()).\
            flatten().frequencies(sorted == True) #flatten = concat

result_2 = js.filter(lambda record: record['overall'] == 2.0).\
            map(lambda x: re.sub('[!@#$.?:;()"]',"",x['reviewText'].lower().strip()).split()).\
            flatten().frequencies(sorted == True) #flatten = concat

count_5 = 0
bag_for_count_5 = result_5.filter(lambda x: x[1] > 10).compute()
for i in bag_for_count_5:
    count_5 += i[1]

count_2 = 0
bag_for_count_2 = result_2.filter(lambda x: x[1] > 10).compute()
for i in bag_for_count_2:
    count_2 += i[1]


result_5 = dict(result_5.filter(lambda x: x[1] > 10).map(lambda x: (x[0], x[1]/count_5)).compute())
result_2 = dict(result_2.filter(lambda x: x[1] > 10).map(lambda x: (x[0], x[1]/count_2)).compute())

In [ ]:
%%time
help_dict = {}
log_list = []
for key, value in result_5.items():
    try:
        help_dict[key] = result_5[key] - result_2[key]
    except:
        #log_list.append(str(result_5) + " не воходит во второй словарь")
        pass

sorted(help_dict.items(), key=lambda x: x[1])[-51:]
sorted(help_dict.items(), key=lambda x: x[1])[:50]

Найти и вывести на экран наименьший элемент в массиве.

In [ ]:
x.min().compute()

В accounts.*.csv найти имя, для которого встречается наибольшее количество положительных значений в столбце amount.

In [ ]:
filename = os.path.join('..\data\accounts', 'accounts.*.csv')
filename

df = dd.read_csv(filename, dtype={'id': int, 'names': str, 'amount': int})

In [ ]:
res = df[df.amount > 0].groupby('names').amount.count().idxmax().compute()
df[df.amount>0].groupby('names').amount.count().idxmax().compute()

Эффективно создать csv файлы, содержащие данные из файлов accounts.*.csv к которым добавлен столбец, в котором содержится количество сотен из положительного значения amount и 0, если значение в amount отрицательное

In [ ]:
df['new'] = df['amount'] // 100
df['new'] = df['new'].where(df['new'] > 0, 0)

In [ ]:
os.mkdir('data_new')
to = os.path.join("data_new", "accounts_new.*.csv")
df.to_csv(to)

Эффективно найти все 10 децилей для распределения значения amount

In [ ]:
da.percentile(df['amount'].values, q=list(range(10, 101, 10))).compute()

Нати 2 последовательно идующие транзакции имеющие наибольшую по модулю разность между значениями amount

In [ ]:
def diff_between_window(df, periods=1):
         before, after = (periods, 0) if periods > 0 else (0, -periods)
         return df.map_overlap(lambda df, periods=1: df.diff(periods),
                           periods, 0, periods=periods)

In [ ]:
tr2 = diff_between_window(df['amount'], 1).abs().idxmax()
tr1 = tr2 - 1

In [ ]:
dask.compute(tr1, tr2)

Для файлов json с отзывами с Amazon подсчиатать количество отзывов для каждой из оценок. Для подсчета количества отзывов использовать Dask.DataFrame

In [ ]:
reviews_json.groupby('overall').overall.count().compute()